# Packages

In [2]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 30.5 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=b9a95d552a53a52b934743c9b450d57b91ebd944d70c6538e451517d3f0ef2ad
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import gc
import string
import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd
import hazm
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', None)
from termcolor import colored
from itertools import chain
#from transformers import BertTokenizer, BertModel
who_am_i = 'Mitra'

# Data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd 'drive/MyDrive/Prose2Poem'

/content/drive/.shortcut-targets-by-id/1csJsmFwDEwLuf4OVVoqnOnSHZeZvxa2a/Prose2Poem


In [8]:
all_data = pd.read_csv('./Data/Parallel Dataset/ProsPoemParallelDataset_augmented.csv')

print('length of augmented cleaned data: ',
      colored(len(all_data), 'blue'))

length of augmented cleaned data:  28820


In [9]:
#Me
train_size = int(0.8 * len(all_data))
train_indices = [i for i in range(train_size)]
val_indices = [i for i in range(train_size, len(all_data))]

In [10]:
len(train_indices)


23056

In [11]:
len(val_indices)

5764

In [12]:
# val_indices = pd.read_pickle('.../validation_indices.pickle')
# train_indices = pd.read_pickle('.../train_indices.pickle')

In [13]:
def clean(t):
    t = re.sub('^ ', '', t)
    t = re.sub(' $', '', t)
    t = re.sub(r' */ *', ' / ', t)
    t = t.replace('\\', '')
    t = re.sub(r' \. *\.', '\.', t)
    t = re.sub(' +\s', ' ', t)

    t = re.sub(' \.$', '\.', t)
    t = re.sub('^ *\. *', '', t)

    t = re.sub('[۱۲۳۴۵۶۷۸۹۰]', '', t)

    return t

all_data.loc[:, 'poetry'] = all_data.loc[:, 'poetry'].apply(lambda x: clean(x))
all_data.loc[:, 'text'] = all_data.loc[:, 'text'].apply(lambda x: clean(x))

In [14]:
all_data.head(2)

,poetry,text
0,دوست دارم که بپوشی رخ همچون قمرت / تا چو خورشید نبینند به هر بام و درت,دوست دارم که چهرۀ چون ماه تابانت را بپوشانی تا مردم تو را مثل خورشید در کوی و برزن نبینند.
1,جرم بیگانه نباشد که تو خود صورت خویش / گر در آیینه ببینی برود دل ز برت,بیگانه در عشق ورزیدن به تو گناهی ندارد زیرا اگر تو هم در آینه به چهرۀ خود بنگری دل از کف می‌دهی.


# PreProcessing + Creating Inputs

In [15]:
normalizer = hazm.Normalizer(persian_numbers=False)

def process_sents(text):

    # separate dot or / from text with
    # one white space
    text = normalizer.normalize(text)

    text = re.sub(r'([\/\.])', r' \1', text)

    # substitute / with sep between mesras
    text = re.sub(r' *\/ *', ' <sep> ', text)

    # substitute any white space with one space
    text = re.sub(r'\s+', ' ', text)

    # add start and end tokens
    text = '<start> ' + text + ' <end>'

    return text


In [16]:
def tokenize(lang):
    # use keras defualt tokenizer
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=''
    )
    # fit on the vocabulary used in text
    lang_tokenizer.fit_on_texts(lang)

    # convert to ids
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                           padding = 'post')

    # add sep to the tokenizer
    #idx_sep = len(lang_tokenizer.index_word.keys())+1#[-1]

    #lang_tokenizer.word_index['<sep>'] = idx_sep
    #lang_tokenizer.index_word[idx_sep] = '<sep>'


    return tensor, lang_tokenizer

In [17]:
def create_load_dataset(df):

    input_lang = df.loc[:, 'text'].values.tolist()
    target_lang = df.loc[:, 'poetry'].values.tolist()

    # preprocess each sentence
    input_lang = [process_sents(text) for text in input_lang]
    target_lang = [process_sents(text) for text in target_lang]

    # create a tensor and tokenizer for each language
    input_tensor, input_lang_tokenizer = tokenize(input_lang)
    target_tensor, target_lang_tokenizer = tokenize(target_lang)

    return input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer

In [18]:
input_tensor, target_tensor,\
input_lang_tokenizer, target_lang_tokenizer = create_load_dataset(all_data)


In [19]:
max_len_input = input_tensor.shape[1]
max_len_target = target_tensor.shape[1]

print('longest sequence and the length of texts: ',
      colored(max_len_input, 'blue'))
print('longest sequence and the length of poetries: ',
      colored(max_len_target, 'blue'))

longest sequence and the length of texts:  70
longest sequence and the length of poetries:  25


# Vocabularies

In [20]:
# lenght of constructed vocabularies:
# 1 for padding
vocab_len_i = len(input_lang_tokenizer.index_word) + 1
print("Plain text vocab has", colored(f"{vocab_len_i:,}", 'green'), "unique words.")

vocab_len_t = len(target_lang_tokenizer.index_word) + 1
print(f"Poetry vocab has", colored(f"{vocab_len_t:,}", 'green'), "unique words.")



Plain text vocab has 18,107 unique words.
Poetry vocab has 11,807 unique words.


In [21]:
def convert(text, poetry):


    print(colored('Text:', 'green'))
    for i in text:
        if i!=0:
            print("%d -----> %s"%(i, input_lang_tokenizer.index_word[i]))

    print(colored('\nPoetry:', 'green'))
    for i in poetry:
        if i!=0:
            print("%d -----> %s"%(i, target_lang_tokenizer.index_word[i]))

In [22]:
print(colored('Text: ', 'blue'), all_data.loc[5, 'text'])
print(colored('Poetry: ', 'blue'), all_data.loc[5, 'poetry'])
convert(input_tensor[5], target_tensor[5])

Text:  به دفعات و مکرر گفته‌ام که چهرۀ زیبای خود را پیش هر کسی آشکار مساز تا هر انسان بی بصیرتی به آن چهره ننگرد.
Poetry:  بارها گفته‌ام این روی به هر کس منمای / تا تأمل نکند دیدۀ هر بی بصرت
Text:
3 -----> <start>
5 -----> به
10902 -----> دفعات
1 -----> و
8419 -----> مکرر
13580 -----> گفته‌ام
6 -----> که
1028 -----> چهرۀ
1003 -----> زیبای
15 -----> خود
8 -----> را
105 -----> پیش
21 -----> هر
29 -----> کسی
510 -----> آشکار
13581 -----> مساز
26 -----> تا
21 -----> هر
73 -----> انسان
86 -----> بی
5626 -----> بصیرتی
5 -----> به
17 -----> آن
604 -----> چهره
5268 -----> ننگرد
2 -----> .
4 -----> <end>

Poetry:
1 -----> <start>
4532 -----> بارها
9452 -----> گفته‌ام
14 -----> این
44 -----> روی
10 -----> به
20 -----> هر
103 -----> کس
9453 -----> منمای
3 -----> <sep>
19 -----> تا
3947 -----> تأمل
538 -----> نکند
2001 -----> دیدۀ
20 -----> هر
41 -----> بی
6839 -----> بصرت
2 -----> <end>


# Creating the Model

In [23]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val =\
input_tensor[train_indices], input_tensor[val_indices],  target_tensor[train_indices], target_tensor[val_indices]

print('Length of train and val:',
      colored(f"{len(input_tensor_train), len(input_tensor_val)}", 'blue'))

Length of train and val: (23056, 5764)


In [24]:
# defining the main parameters of the model
# and the inputs

len_data = len(input_tensor_train)
batch_s = 256
steps_per_epoch = len_data // batch_s
embedding_dim = 256
units = 1024

In [25]:
# create the dataset and shuffle all
len_data_train = len(input_tensor_train)
len_data_test = len(target_tensor_val)

# creat the datasets and put them in batches

train_batches = tf.data.Dataset.from_tensor_slices((
    np.array(input_tensor_train.tolist(), dtype='int32'),
     np.array(target_tensor_train.tolist(), dtype='int32')
)).shuffle(len_data_train).batch(batch_s, drop_remainder=True)



In [26]:
input_batch_sample, target_batch_sample = next(iter(train_batches))

print('A sample of text(input) batch: \n',
      colored(input_batch_sample, 'blue'))

print('\nA sample of poetry(target) batch: \n',
      colored(target_batch_sample, 'blue'))



A sample of text(input) batch: 
 tf.Tensor(
[[   3   24 8379 ...    0    0    0]
 [   3  128  487 ...    0    0    0]
 [   3   12   27 ...    0    0    0]
 ...
 [   3  110    9 ...    0    0    0]
 [   3   16   18 ...    0    0    0]
 [   3 3468  581 ...    0    0    0]], shape=(256, 70), dtype=int32)

A sample of poetry(target) batch: 
 tf.Tensor(
[[   1  282  217 ...    0    0    0]
 [   1  139 1716 ...    0    0    0]
 [   1  220   25 ...    0    0    0]
 ...
 [   1   16 2841 ...    0    0    0]
 [   1   80   72 ...    0    0    0]
 [   1  226  872 ...    0    0    0]], shape=(256, 25), dtype=int32)


In [27]:
class Encoder(tf.keras.Model):

    def __init__(self, vocab_size, embedding_dim,
                 enc_units, batch_s, ):

        # change the primary model from keras
        super(Encoder, self).__init__()

        # define the parameters
        self.batch_s = batch_s
        self.enc_units = enc_units
        self.embeddings = tf.keras.layers.Embedding(vocab_size,
                                                    embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')


    def call(self, x, hidden):
        # first part of the model
        # calling the embeddings and giving them
        # to the gru
        x = self.embeddings(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state


    def initialize_hidden_state(self):
        # the initial state of the hidden states
        # start with zeros
        return tf.zeros((self.batch_s, self.enc_units))

In [28]:
encoder = Encoder(vocab_len_i, embedding_dim, units, batch_s)

In [29]:
# initialize the hs
sample_hidden_states = encoder.initialize_hidden_state()
# get the output of the encoder
sample_encoder_output, sample_hidden_states_encoder = encoder(input_batch_sample, sample_hidden_states)

print('Encoder hidden states shapes:',
      colored(sample_hidden_states_encoder.shape, 'blue'))
print('Encoder output shape:',
      colored(sample_encoder_output.shape, 'blue'))

Encoder hidden states shapes: (256, 1024)
Encoder output shape: (256, 70, 1024)


In [30]:
print('Hidden states after being processed in gru:\n',
      colored(sample_hidden_states_encoder, 'blue'))

print('\nEncoder output sample:\n', colored(sample_encoder_output, 'blue'))

Hidden states after being processed in gru:
 tf.Tensor(
[[-0.01696949  0.02688834 -0.02156566 ... -0.00509517  0.00107555
  -0.02101996]
 [-0.01696949  0.02688834 -0.02156566 ... -0.00509517  0.00107555
  -0.02101996]
 [-0.01696949  0.02688834 -0.02156566 ... -0.00509517  0.00107555
  -0.02101996]
 ...
 [-0.01696949  0.02688834 -0.02156566 ... -0.00509518  0.00107555
  -0.02101996]
 [-0.01696949  0.02688834 -0.02156566 ... -0.00509517  0.00107555
  -0.02101996]
 [-0.01696949  0.02688834 -0.02156566 ... -0.00509517  0.00107555
  -0.02101996]], shape=(256, 1024), dtype=float32)

Encoder output sample:
 tf.Tensor(
[[[-3.74829234e-03  1.00068878e-02 -7.89913058e-04 ... -1.32240565e-03
    3.14204372e-03 -2.22523790e-03]
  [-6.05875952e-03  3.70479957e-03  7.06838211e-03 ... -6.75688498e-03
    6.06072461e-03 -2.47711758e-03]
  [-3.65584344e-03 -3.07724666e-04  4.79982747e-03 ...  4.74262750e-03
    3.28678824e-03 -3.03384103e-03]
  ...
  [-1.69694927e-02  2.68883388e-02 -2.15656571e-02 ...

In [31]:
class Decoder(tf.keras.Model):
    # create the decoder side

    def __init__(self, vocab_size, embedding_dim,
                 decoder_units, batch_s, ):

        # take and change the keras.model
        super(Decoder, self).__init__()
        # parameters
        self.batch_s = batch_s
        self.decoder_units = decoder_units
        self.embeddings = tf.keras.layers.Embedding(vocab_size, embedding_dim)

        self.gru = tf.keras.layers.GRU(self.decoder_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        # fully connected
        self.fc = tf.keras.layers.Dense(vocab_size,
                                        activation='softmax')

        self.attention = BahdanauAttention(self.decoder_units)

    def call(self, x, hidden, encoder_output):
        # construct the decoder

        # x shape = (batch_s, 1)
        # one id for each word from the target

        # encoder output = (batch_s, max_len, hidden_states_s)
        context_vector = self.attention(query=hidden,
                                                           value=encoder_output)

        # we expand the ids into embedding vectors
        # x = (batch_s, 1, embedding_dim)
        x = self.embeddings(x)

        # concatenating hidden states and the context
        # vector
        x = tf.concat([tf.expand_dims(context_vector, 1), x],
                      axis=-1)

        # give both attention and embeddings to gru
        output, state = self.gru(x)

        # output = (batch_size, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # batch_s, vocab_size
        x = self.fc(output)

        return x, state



In [32]:
class BahdanauAttention(tf.keras.layers.Layer):

    def __init__(self, units):
        super(BahdanauAttention, self).__init__()

        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, value):
        # query = batch_s, hidden_states - from decoder
        # query_with_time = batch_s, 1, hidden_states
        # values = batch_s, max_len_input, hidden_states

        # adding one dimention to take time into account
        query_with_time_axis = tf.expand_dims(query, 1)


        # combining values and queries
        # attention scores = batch_s, max_len, 1
        attention_score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(value)
        ))

        # getting a softmax to choose the weights for
        # each position in input
        # batch_s, max_len_input, 1
        attention_weights = tf.nn.softmax(attention_score, axis=1)

        # after multiplication and summing:
        # context_vector = batch_s, hidden_s
        context_vector = attention_weights * value
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector

In [33]:
decoder = Decoder(vocab_len_t, embedding_dim, units, batch_s)


sample_decoder_output, states = decoder(x = tf.random.uniform((batch_s, 1)),
                                        hidden = sample_hidden_states_encoder,
                                        encoder_output = sample_encoder_output)

# Train

In [34]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

def loss_function(real, pred):

    # first mask the ones that are not paddings
    mask = tf.math.logical_not(tf.math.equal(real, 0))

    # apply the loss on the whole sequence
    loss_ = loss_object(real, pred)

    # make the mask datatype the same as loss
    mask = tf.cast(mask, dtype=loss_.dtype)

    loss_ = loss_ * mask

    # return the mean of all words
    return tf.reduce_mean(loss_)


In [35]:


# Switching from eager execution to graph execution

@tf.function
def train_step(input, target, encoder_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        # get to the encoder
        encoder_output, encoder_hidden_states = encoder(input, encoder_hidden)

        # start the decoder
        decoder_hidden_states = encoder_hidden_states

        # give <s> to all as the first word
        decoder_input = \
        tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * batch_s, 1)

    # Teacher Forcing
    # start from the first word and continue
    # till the end of the sequence

        for t in range(1, target.shape[1]):

            # give
            # 1. decoder input that starts with <start>
            # 2. decoder hidden states
            # 3. encoder output
            # to the decoder

            Seq2Seq_logits, decoder_hidden_states = decoder(
                decoder_input, decoder_hidden_states, encoder_output
            )

            # ------------------------------------
            # calculate the loss at time step t
            loss += loss_function(target[:, t], Seq2Seq_logits)

            # change the decoder input to the target token of
            # this time step for
            # Teacher Forcing
            decoder_input = tf.expand_dims(target[:, t], axis=1)

    # get the mean loss
    batch_loss = (loss / int(target.shape[1]))

    # get the variables that have been changed
    variables = encoder.trainable_variables + decoder.trainable_variables

    # calculate the gradients based on the loss
    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [41]:
gc.collect()

53

In [42]:
# alpha = 0.6
epochs =
start_id = '<start>'

for epoch in range(epochs):

    print("epoch ", colored(epoch, 'blue'))
    start = time.time()

    encoder_hidden_state = encoder.initialize_hidden_state()
    total_loss = 0

    for (batch, (input, target)) in enumerate(train_batches.take(steps_per_epoch)):

        batch_loss = train_step(input, target, encoder_hidden_state)
        total_loss += batch_loss

        gc.collect()

        if batch%50==0:
            print('batch ', colored(batch, 'green'),
                  f' Loss {batch_loss.numpy():.4f}')

    # checkpoint.save(file_prefix=checkpoint_prefix)
    print(f'Time taken: {time.time() - start:.2f} seconds')

epoch  0
batch  0  Loss 4.0598
batch  50  Loss 3.9139
Time taken: 201.91 seconds
epoch  1
batch  0  Loss 3.7582
batch  50  Loss 3.7787
Time taken: 129.42 seconds


# Normal Evaluation

In [43]:
def evaluate(sentence):


    # attention_plot = np.zeros((max_len_target,
      #                         max_len_input))

    # preprocessing every sentence before giving
    # them to the model
    sentence = process_sents(sentence)
    # converting str to ids and padding and creating a tensor
    # from all
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_len_input,
                                                          padding='post')
    inputs = tf.convert_to_tensor(inputs)

    # start constructing the output string
    output = ''

    hidden_state = [tf.zeros((1, units))]
    encoder_output, encoder_hidden_state = encoder(inputs, hidden_state)

    decoder_hidden_state = encoder_hidden_state

    decoder_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)


    for t in range(max_len_target):
        predictions, decoder_hidden_state = decoder(
            decoder_input, decoder_hidden_state,
            encoder_output
        )


        # sotring for plot
        # attention_weights = tf.reshape(attention_weights, (-1, ))
        # attention_plot[t] = attention_weights.numpy()
        # predict the most probable token
        predicted_id = tf.argmax(predictions[0]).numpy()

        # add this token to the previous ones
        output = output + target_lang_tokenizer.index_word[predicted_id] + ' '

        # is it over?
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return output, sentence

        # give the prediction to continue predicting
        # next tokens
        decoder_input = tf.expand_dims([predicted_id], 0)


    return output, sentence

In [44]:
def transform(text):
    output, text = evaluate(sentence=text)

    print('Text: ', text)
    print('Generate poetry:', output)


    # attention_plot = attention_plot[:len(output.split(' ')),
     #                             :len(text.split(' '))]
    # plot_attention(attention_plot, text.split(' '), output.split(' '))

In [45]:
transform('با این توصیف عشاق بی عقل و بدون هدف خاص زندگی می کنند و دارای هیچ هدف و مغزی نیستند تا اینکه به جهنم می رسند و به هیچ جایگاه دنیوی و واقعی دست پیدا نمی کنند')

Text:  <start> با این توصیف عشاق بی‌عقل و بدون هدف خاص زندگی می‌کنند و دارای هیچ هدف و مغزی نیستند تا اینکه به جهنم می‌رسند و به هیچ جایگاه دنیوی و واقعی دست پیدا نمی‌کنند <end>
Generate poetry: تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا 


# evaluate a dataset

In [46]:
def evaluate_dataset(df):

    generated_p = []

    df = df.reset_index(drop=True)
    for r in range(len(df)):

        try:
            # attention_plot = np.zeros((max_len_target,
            #                         max_len_input))

            # preprocessing every sentence before giving
            # them to the model
            sentence = process_sents(df.loc[r, 'text'])
            # converting str to ids and padding and creating a tensor
            # from all
            inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
            inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=max_len_input,
                                                                padding='post')
            inputs = tf.convert_to_tensor(inputs)

            # start constructing the output string
            output = ''

            hidden_state = [tf.zeros((1, units))]
            encoder_output, encoder_hidden_state = encoder(inputs, hidden_state)

            decoder_hidden_state = encoder_hidden_state

            decoder_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)


            for t in range(max_len_target):
                predictions, decoder_hidden_state = decoder(
                    decoder_input, decoder_hidden_state,
                    encoder_output
                )


                # sotring for plot
                # attention_weights = tf.reshape(attention_weights, (-1, ))
                # attention_plot[t] = attention_weights.numpy()
                # predict the most probable token
                predicted_id = tf.argmax(predictions[0]).numpy()

                # add this token to the previous ones
                output = output + target_lang_tokenizer.index_word[predicted_id] + ' '

                # is it over?
                if target_lang_tokenizer.index_word[predicted_id] == '<end>':
                    break

                # give the prediction to continue predicting
                # next tokens
                decoder_input = tf.expand_dims([predicted_id], 0)

            generated_p.append(output)

        except:
            print(r)
            print(df.loc[r, 'text'])

            generated_p.append(None)

    df_output = pd.concat([df, pd.Series(generated_p)],
                                axis = 1)

    df_output.columns = ['poetry_ground_truth',
                        'text',
                        'poetry_generated_Seq2Seq_with_Att']


    return df_output

In [47]:
val_indices

[23056,
 23057,
 23058,
 23059,
 23060,
 23061,
 23062,
 23063,
 23064,
 23065,
 23066,
 23067,
 23068,
 23069,
 23070,
 23071,
 23072,
 23073,
 23074,
 23075,
 23076,
 23077,
 23078,
 23079,
 23080,
 23081,
 23082,
 23083,
 23084,
 23085,
 23086,
 23087,
 23088,
 23089,
 23090,
 23091,
 23092,
 23093,
 23094,
 23095,
 23096,
 23097,
 23098,
 23099,
 23100,
 23101,
 23102,
 23103,
 23104,
 23105,
 23106,
 23107,
 23108,
 23109,
 23110,
 23111,
 23112,
 23113,
 23114,
 23115,
 23116,
 23117,
 23118,
 23119,
 23120,
 23121,
 23122,
 23123,
 23124,
 23125,
 23126,
 23127,
 23128,
 23129,
 23130,
 23131,
 23132,
 23133,
 23134,
 23135,
 23136,
 23137,
 23138,
 23139,
 23140,
 23141,
 23142,
 23143,
 23144,
 23145,
 23146,
 23147,
 23148,
 23149,
 23150,
 23151,
 23152,
 23153,
 23154,
 23155,
 23156,
 23157,
 23158,
 23159,
 23160,
 23161,
 23162,
 23163,
 23164,
 23165,
 23166,
 23167,
 23168,
 23169,
 23170,
 23171,
 23172,
 23173,
 23174,
 23175,
 23176,
 23177,
 23178,
 23179,
 23180,


In [48]:

output_df = evaluate_dataset(all_data.loc[val_indices])
output_df

,poetry_ground_truth,text,poetry_generated_Seq2Seq_with_Att
0,شد از خویش و پیوند و سرشت / کردهای در چشم او هر خوب خبر,انان را از اعمال و عادات و طبیعتشان بردهای یعنی خلاصشان کردهای. و چیز خوب و زیبا را در نظر انان زشت و بد جلوه دادهاست. به عبارت دیگر از حیز محسوسات خارج و فرار کردهای.,تا که که در تو را <end>
1,بردهای از خویش و پاک و سرشت / کردهای در چشم او هر خوب روی,خود را از خویشتن و خویشاوندان و اقوام بردهای یعنی خلاصشان کردهای. و چیز خوب و زیبا را در نظر خویش زشت و بد جلوه دادهای. به عبارت دیگر از دیار خود گریخته و فرار کردهای.,تا که که در تو را <end>
2,هر چه محسوس است او رد میکند / و یا ناپیداست مسند میکند,چنین عارف عارفی هر چیز ضروری و قریب به عالم باطنی را پس زده و به عالم غیب تکیه میکند.,تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا
3,هر چه محسوس است او رد میکند / و تو ناپیداست مسند میکند,چنین و عارفی هر چیز محسوس و متعلق به ان است را پس میزند و به عالم غیب رجوع میکند.,تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا
4,عشق او پیدا و معشوقش نهان / هم بیرون شده او در جهان,چنین کسی عشق و دوستی اش اشکار و معشوقش پنهان است. به همین مناسبت حضرت علی در بیرون از این بند پنهان است ولی میل به عشق او در همین است..,تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا
...,...,...,...
5759,ابر آب داد و درختان چمن را / شاخ برهنه پیرهن گیاه کرد,سپس تا مدتها درختان مرده را خشک کرده و بار دیگر درختان مرده زمستانی را ابیاری کرده و لباس فصل بهار را که پر از گل و گیاه است بر تن خود میکنند.,تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا
5760,ابر ار داد بیخ درختان مرده را / شاخ برهنه شاخ تازه کرد,ابرها تا حدی درختان مرده را قطع میکند و بار دیگر درختان خشک زمستانی را خشک کرده و لباسهای فصل بهار را که پر از گل و گیاه است بر تن انان میپوشاند.,تا که که در تو را <end>
5761,توحیدگوی او نه گفت و بس / هر گل که غنچه بر شاخسار کرد,تنها انسانها نیستند که خداوند را یکتا میدانند و ذکر او را میگویند ؛ بلکه تمام موجودات عالم به تسبیح او میپردازند. هر بلبلی که روی درختان نماز میخواند درواقع به ذکر خدا مشغول است.,تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا که که در تو <sep> تا
5762,دیدار یار چه دانی چه ذوق دارد / هر که در بیابان بر خود ببارد,اما میدانید که کار کسی که تاکنون غایب بوده چقدر سرگرم کننده است و همه را بر سر ذوق میاورد,تا که که در تو را <end>


In [50]:
output_df.to_csv(f'./Data/Seq2Seq_with_att_{epochs}_epochs_{batch_s}_batch_s_{embedding_dim}_embedding_dim_{units}_units_.csv',
                 index=False)